## Method Details

The tutorial will cover several methods for implementing zero-shot prompting:

1. **Direct Task Specification**: Crafting prompts that clearly define the task without examples.
2. **Role-Based Prompting**: Assigning specific roles to the AI to guide its responses.
3. **Format Specification**: Providing output format guidelines in the prompt.
4. **Multi-step Reasoning**: Breaking down complex tasks into simpler zero-shot steps.
5. **Comparative Analysis**: Evaluating different zero-shot prompt structures for the same task.

In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv()

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [2]:
def create_chain(prompt_template):
    """"
    Create a Langchain chain with given prompt.
    Args:
        Prompt_template(str): The prompt template string.

    Returns:
        LLMChain : A Langchain chain object    
    """
    prompt=PromptTemplate.from_template(prompt_template)
    return prompt | llm

# Direct Task Specification
 We will learn how to craft prompts that clearly define task without providing example.

In [3]:
direct_task_prompt='''
        Classify the sentiment of the following text as positive,negative or neutral.
        Do not explain reasoning, just provide the classification.
        Text: {text}
        Sentiment:
        '''
direct_task_prompt=create_chain(direct_task_prompt)

# Example
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result=direct_task_prompt.invoke({'text':text}).content
    print(f"Text:{text}")
    print(f"Sentiment:{result}")

Text:I absolutely loved the movie! The acting was superb.
Sentiment:Positive
Text:The weather today is quite typical for this time of year.
Sentiment:Neutral
Text:I'm disappointed with the service I received at the restaurant.
Sentiment:Negative


# Format Specification
Provide output format guidelines in the prompt, that can help structure the AI's response

In [4]:
format_spec_prompt="""
Generate a short news article about {topic}.
Structure your response in the following format:
Headline: [A catchy headline for the article]
Lead: [A brief introductory paragraph summarizing the key points]
Body: [2-3 short paragraphs providing more details]
Conclusion: [A concluding sentence or call to action]
"""

format_spec_prompt=create_chain(format_spec_prompt)

# Test
topic="The discovery of Black Hole"
result=format_spec_prompt.invoke({'topic':topic}).content
print(result)


Headline:  Cosmic Heavyweight Discovered:  New Black Hole Shakes Up Galactic Understanding!

Lead: Astronomers have announced the discovery of a previously unknown black hole, significantly closer to Earth than any previously identified stellar-mass black hole.  This remarkable find, located just 1,500 light-years away in the constellation Telescopium, challenges current models of stellar evolution and offers unprecedented opportunities for studying these enigmatic cosmic objects.

Body:  The black hole, dubbed Gaia BH1, was detected using data from the European Space Agency's Gaia satellite and confirmed through follow-up observations with other telescopes.  Unlike most known black holes, Gaia BH1 is surprisingly quiet, showing no signs of actively devouring surrounding material. This "dormant" nature makes it particularly intriguing to scientists, as it provides a unique window into the lifecycle of black holes. The discovery was made by a team of international astronomers who meticu

# Multi-step Reasoning
For complex tasks, we can break question down into simpler zero-shot steps. This improves overall performance of the model.

In [5]:
multi_step_prompt="""
    Analyze the following text for its  main arguments, supporting evidence and potential counterarguments.
    Provide your analysis in the following steps:
    1. Main Arguments: Identify and state the primary claim or thesis
    2. Supporting Evidence: List the key points or evidence used to support the main argument.
    3. Potential  Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.
    Text: {text}
    Analysis:
"""

multi_step_chain=create_chain(multi_step_prompt)

# Test
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result=multi_step_chain.invoke({'text':text}).content
print(result)

**Analysis:**

**1. Main Arguments:** The primary claim is that the environmental impact of electric vehicles (EVs) is more complex than often portrayed, and their contribution to climate change mitigation is not guaranteed.  The text doesn't outright oppose EVs, but it argues against the simplistic narrative of them being a straightforward solution.

**2. Supporting Evidence:**

* **Battery production's negative impacts:** The text highlights the significant environmental damage caused by mining operations necessary for EV battery production. This includes habitat destruction and water pollution. This points to a significant upfront environmental cost.
* **Electricity source dependence:** The text emphasizes that the overall carbon footprint of EVs is highly dependent on the source of electricity used to charge them. If the electricity comes from fossil fuels, the reduction in greenhouse gas emissions compared to gasoline vehicles might be minimal or even nonexistent. This highlights 

# Comparative Analysis

In [6]:
def compare_prompts(task, prompt_templates):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"Task: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chain(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

task = "Explain conciesly the concept of blockchain technology"

prompt_templates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

compare_prompts(task, prompt_templates)

Task: Explain conciesly the concept of blockchain technology

Basic Prompt Result:
Blockchain is a shared, immutable ledger of transactions verified and secured by cryptography.  It's a decentralized database, meaning no single entity controls it, making it transparent and resistant to tampering.

--------------------------------------------------

Structured Prompt Result:
1. **Definition:** Blockchain is a decentralized, immutable ledger of transactions verified and secured by cryptography.  It's a shared, replicated database accessible to all participants.

2. **Key features:** Decentralization (no single point of failure), immutability (records cannot be altered), transparency (all transactions are visible), security (cryptographic hashing ensures integrity).

3. **Real-world applications:** Cryptocurrencies (Bitcoin), supply chain management (tracking goods), digital identity verification, voting systems, healthcare data management.

4. **Potential impact on industries:** Increase